In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import yfinance as yf
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [3]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [4]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [5]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,7,1)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df
hourly_df
ohlc_df

,time,open,high,low,close
0,2025-06-30 23:00:00,1.17797,1.17798,1.17774,1.17797
1,2025-06-30 23:01:00,1.17797,1.17807,1.17794,1.17795
2,2025-06-30 23:02:00,1.17795,1.17801,1.17795,1.17799
3,2025-06-30 23:03:00,1.17799,1.17800,1.17791,1.17797
4,2025-06-30 23:04:00,1.17797,1.17801,1.17791,1.17791
...,...,...,...,...,...
51936,2025-08-20 01:29:00,1.16483,1.16486,1.16481,1.16484
51937,2025-08-20 01:30:00,1.16484,1.16485,1.16475,1.16479
51938,2025-08-20 01:31:00,1.16479,1.16488,1.16477,1.16484
51939,2025-08-20 01:32:00,1.16484,1.16486,1.16475,1.16480


In [6]:
#ohlc4
daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
#sma
daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
daily_df

,time,open,high,low,close,ohlc,sma_1,sma_14,high_20,low_20
0,2025-07-01,1.17868,1.18298,1.17615,1.18052,1.179582,1.179582,NaN,NaN,NaN
1,2025-07-02,1.18056,1.18101,1.17470,1.17982,1.179023,1.179023,NaN,NaN,NaN
2,2025-07-03,1.17983,1.18101,1.17172,1.17573,1.177072,1.177072,NaN,NaN,NaN
3,2025-07-04,1.17562,1.17876,1.17502,1.17736,1.176690,1.176690,NaN,NaN,NaN
4,2025-07-07,1.17771,1.17903,1.16868,1.17083,1.174063,1.174063,NaN,NaN,NaN
5,2025-07-08,1.17076,1.17654,1.16826,1.17250,1.172015,1.172015,NaN,NaN,NaN
6,2025-07-09,1.17243,1.17294,1.16896,1.17222,1.171638,1.171638,NaN,NaN,NaN
7,2025-07-10,1.17173,1.17497,1.16626,1.17000,1.170740,1.170740,NaN,NaN,NaN
8,2025-07-11,1.16999,1.17141,1.16647,1.16852,1.169097,1.169097,NaN,NaN,NaN
9,2025-07-14,1.16580,1.16977,1.16543,1.16653,1.166882,1.166882,NaN,NaN,NaN


In [7]:
def entring_stoploss_profit(row, hourly_df) :
    if row["sma_1"] > row['sma_14'] and row["sma_11"] > row['sma_144'] :
        return "buy"
    elif row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_144'] :
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, hourly_df = hourly_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
18760,sell,2025-07-18 00:00:00
21636,buy,2025-07-22 00:00:00
23075,buy,2025-07-23 00:00:00
24514,buy,2025-07-24 00:00:00
27870,sell,2025-07-28 08:00:00
28829,sell,2025-07-29 00:00:00
30268,sell,2025-07-30 00:00:00
31707,sell,2025-07-31 00:00:00
33146,sell,2025-08-01 00:00:00
36742,sell,2025-08-05 12:00:00


In [8]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close"
})


In [9]:
print(merged_df.columns.tolist())
merged_df

['time', 'open', 'high', 'low', 'close', 'hourly_open', 'hourly_high', 'hourly_low', 'hourly_close', 'ohlc_x', 'sma_11', 'sma_144', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'ohlc_y', 'sma_1', 'sma_14', 'high_20', 'low_20', 'signal', 'is_first_signal']


,time,open,high,low,close,hourly_open,hourly_high,hourly_low,hourly_close,ohlc_x,...,daily_high,daily_low,daily_close,ohlc_y,sma_1,sma_14,high_20,low_20,signal,is_first_signal
0,2025-06-30 23:00:00,1.17797,1.17798,1.17774,1.17797,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,2025-06-30 23:01:00,1.17797,1.17807,1.17794,1.17795,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,2025-06-30 23:02:00,1.17795,1.17801,1.17795,1.17799,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,2025-06-30 23:03:00,1.17799,1.17800,1.17791,1.17797,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,2025-06-30 23:04:00,1.17797,1.17801,1.17791,1.17791,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51936,2025-08-20 01:29:00,1.16483,1.16486,1.16481,1.16484,1.16443,1.16491,1.16314,1.16388,1.16409,...,1.16491,1.16275,1.16328,1.163843,1.163843,1.163343,1.166856,1.158269,None,NaN
51937,2025-08-20 01:30:00,1.16484,1.16485,1.16475,1.16479,1.16443,1.16491,1.16314,1.16388,1.16409,...,1.16491,1.16275,1.16328,1.163843,1.163843,1.163343,1.166856,1.158269,None,NaN
51938,2025-08-20 01:31:00,1.16479,1.16488,1.16477,1.16484,1.16443,1.16491,1.16314,1.16388,1.16409,...,1.16491,1.16275,1.16328,1.163843,1.163843,1.163343,1.166856,1.158269,None,NaN
51939,2025-08-20 01:32:00,1.16484,1.16486,1.16475,1.16480,1.16443,1.16491,1.16314,1.16388,1.16409,...,1.16491,1.16275,1.16328,1.163843,1.163843,1.163343,1.166856,1.158269,None,NaN


In [14]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = (data['high'] - data["low_20"]) # example SL
        volume = 50000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = (data['high_20'] - data["low_20"]) # example SL
        volume = 50000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["sma_11"] <= data["sma_144"]:
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data["sma_11"] >= data["sma_144"]:
            orders.close_trade(trade)

In [15]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,50000,2025-07-18 00:00:00,1.1595,2025-07-18 04:00:00,1.16246,0,0,{},-148.0,-3.5,-151.5,-151.5,9848.5
1,closed,EURUSD,buy,50000,2025-07-22 00:00:00,1.16894,2025-07-25 00:00:00,1.17442,0,0,{},274.0,-3.5,270.5,119.0,10119.0
2,closed,EURUSD,sell,50000,2025-07-28 08:00:00,1.17474,2025-08-01 12:00:00,1.14066,0,0,{},1704.0,-3.5,1700.5,1819.5,11819.5
3,closed,EURUSD,sell,50000,2025-08-05 12:00:00,1.15368,2025-08-05 16:00:00,1.15372,0,0,{},-2.0,-3.5,-5.5,1814.0,11814.0
4,closed,EURUSD,buy,50000,2025-08-07 00:00:00,1.16589,2025-08-08 12:00:00,1.16371,0,0,{},-109.0,-3.5,-112.5,1701.5,11701.5
5,closed,EURUSD,buy,50000,2025-08-11 04:00:00,1.16474,2025-08-11 12:00:00,1.16548,0,0,{},37.0,-3.5,33.5,1735.0,11735.0
6,closed,EURUSD,buy,50000,2025-08-12 16:00:00,1.16425,2025-08-14 12:00:00,1.16685,0,0,{},130.0,-3.5,126.5,1861.5,11861.5
7,closed,EURUSD,buy,50000,2025-08-15 12:00:00,1.16822,2025-08-18 12:00:00,1.16819,0,0,{},-1.5,-3.5,-5.0,1856.5,11856.5


In [16]:
mm = bt.plot_balance()
display(mm)

In [17]:
evaluate_backtest(bt.trades)

📊 Backtest Evaluation
Daily Drawdown (worst per day, intrabar): -384.50
Max Equity Drawdown (portfolio): -384.50
worst trades (trade MAE): [np.float64(-384.49999999999875), np.float64(-52.000000000007596), np.float64(739.9999999999962), np.float64(nan), np.float64(-239.5000000000036), np.float64(nan), np.float64(-57.49999999999922), np.float64(-131.00000000000333)]
max_intrabar_drawdown: -384.50
biggest_profit: 1704.0
equity_daily_drawdown: -148.0
equity_max_drawown: -118.0
avg_win: 536.25
avg_loss: -65.12
count_profit_trades: 4
count_loss_trades: 4
Win Rate1: 50.00%
rrr: 8.23


,order_type,profit
0,buy,330.5
1,sell,1554.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown
1,closed,EURUSD,buy,50000,2025-07-22 00:00:00,1.16894,2025-07-25 00:00:00,1.17442,0,0,...,274.0,-3.5,270.5,119.0,10119.0,2025-07-22,2025-07-25 00:00:00,-52.0,119.0,0.0
2,closed,EURUSD,sell,50000,2025-07-28 08:00:00,1.17474,2025-08-01 12:00:00,1.14066,0,0,...,1704.0,-3.5,1700.5,1819.5,11819.5,2025-07-28,2025-08-01 12:00:00,740.0,1819.5,0.0
5,closed,EURUSD,buy,50000,2025-08-11 04:00:00,1.16474,2025-08-11 12:00:00,1.16548,0,0,...,37.0,-3.5,33.5,1735.0,11735.0,2025-08-11,2025-08-11 12:00:00,NaN,1819.5,-84.5
6,closed,EURUSD,buy,50000,2025-08-12 16:00:00,1.16425,2025-08-14 12:00:00,1.16685,0,0,...,130.0,-3.5,126.5,1861.5,11861.5,2025-08-12,2025-08-14 12:00:00,-57.5,1861.5,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown
0,closed,EURUSD,sell,50000,2025-07-18 00:00:00,1.1595,2025-07-18 04:00:00,1.16246,0,0,...,-148.0,-3.5,-151.5,-151.5,9848.5,2025-07-18,2025-07-18 04:00:00,-384.5,-151.5,0.0
3,closed,EURUSD,sell,50000,2025-08-05 12:00:00,1.15368,2025-08-05 16:00:00,1.15372,0,0,...,-2.0,-3.5,-5.5,1814.0,11814.0,2025-08-05,2025-08-05 16:00:00,NaN,1819.5,-5.5
4,closed,EURUSD,buy,50000,2025-08-07 00:00:00,1.16589,2025-08-08 12:00:00,1.16371,0,0,...,-109.0,-3.5,-112.5,1701.5,11701.5,2025-08-07,2025-08-08 12:00:00,-239.5,1819.5,-118.0
7,closed,EURUSD,buy,50000,2025-08-15 12:00:00,1.16822,2025-08-18 12:00:00,1.16819,0,0,...,-1.5,-3.5,-5.0,1856.5,11856.5,2025-08-15,2025-08-18 12:00:00,-131.0,1861.5,-5.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,balance,date,time,intrabar_drawdown,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,sell,50000,2025-07-18 00:00:00,1.1595,2025-07-18 04:00:00,1.16246,0,0,...,9848.5,2025-07-18,2025-07-18 04:00:00,-384.5,-151.5,0.0,4,0,2025-07,2025
1,closed,EURUSD,buy,50000,2025-07-22 00:00:00,1.16894,2025-07-25 00:00:00,1.17442,0,0,...,10119.0,2025-07-22,2025-07-25 00:00:00,-52.0,119.0,0.0,1,0,2025-07,2025
2,closed,EURUSD,sell,50000,2025-07-28 08:00:00,1.17474,2025-08-01 12:00:00,1.14066,0,0,...,11819.5,2025-07-28,2025-08-01 12:00:00,740.0,1819.5,0.0,0,8,2025-08,2025
3,closed,EURUSD,sell,50000,2025-08-05 12:00:00,1.15368,2025-08-05 16:00:00,1.15372,0,0,...,11814.0,2025-08-05,2025-08-05 16:00:00,NaN,1819.5,-5.5,1,12,2025-08,2025
4,closed,EURUSD,buy,50000,2025-08-07 00:00:00,1.16589,2025-08-08 12:00:00,1.16371,0,0,...,11701.5,2025-08-07,2025-08-08 12:00:00,-239.5,1819.5,-118.0,3,0,2025-08,2025
5,closed,EURUSD,buy,50000,2025-08-11 04:00:00,1.16474,2025-08-11 12:00:00,1.16548,0,0,...,11735.0,2025-08-11,2025-08-11 12:00:00,NaN,1819.5,-84.5,0,4,2025-08,2025
6,closed,EURUSD,buy,50000,2025-08-12 16:00:00,1.16425,2025-08-14 12:00:00,1.16685,0,0,...,11861.5,2025-08-12,2025-08-14 12:00:00,-57.5,1861.5,0.0,1,16,2025-08,2025
7,closed,EURUSD,buy,50000,2025-08-15 12:00:00,1.16822,2025-08-18 12:00:00,1.16819,0,0,...,11856.5,2025-08-15,2025-08-18 12:00:00,-131.0,1861.5,-5.0,4,12,2025-08,2025


max_drawdown -118.0
